In [1]:
import random
import librosa
import os
import numpy as np
import pandas as pd
from datetime import datetime
import datetime
from ketos.data_handling import selection_table as sl
import ketos.data_handling.database_interface as dbi
from ketos.data_handling.database_interface import AudioWriter, create_database
from ketos.data_handling.parsing import load_audio_representation
import ketos.audio.audio_loader as al
from ketos.audio.spectrogram import MagSpectrogram


2022-01-13 09:36:37.961190: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-01-13 09:36:37.964899: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-01-13 09:36:37.965116: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (cdr767.int.cedar.computecanada.ca): /proc/driver/nvidia/version does not exist
2022-01-13 09:36:37.968673: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
/home/sadman/ketos_dl/lib/python3.7/site-packages/keras/optimizer_v2/adam.py:105: UserWar

In [2]:
# root_path='/home/sadmans/KW_detector_multiclass/' # Orca-VM
# root_path='/home/sadman/KW_detector_multiclass/' # DL Training pc
# root_path = '/home/sadman/projects/ctb-ruthjoy/sadman/Projects/KW_detector_multiclass/'  # Cedar
root_path='../../' 

# Load spectrogram config file
spec_cfg = load_audio_representation(root_path+'code/create_db/spec_config.json', name="spectrogram")
print("spec_cfg:", spec_cfg)

path_dict = {'train_annot': root_path+'annotations/train/',
             'test_annot': root_path+'annotations/test/',
             'audio_data_dir': '/data/audio',
             'database_save_filename': root_path+'saved_database/test_ds_multiclass_original.h5'
             }

select_step, select_min_overlap = spec_cfg['duration'], 1.0


spec_cfg: {'duration': 5.0, 'rate': 10000, 'window': 0.051, 'step': 0.01955, 'freq_min': 0, 'freq_max': 6000, 'window_func': 'hamming', 'normalize_wav': True, 'type': 'MagSpectrogram', 'transforms': [{'name': 'reduce_tonal_noise'}, {'name': 'normalize', 'mean': 0.0, 'std': 1.0}]}


In [3]:
def create_jasco_roberts_kw_db():
    """ Load train true positive (KW), and false positive (Non-KW) annotations.
        Preprocess the annotation by adding appropriate path.
        Finally, create and save hdf5 file
    """

    tp = pd.read_csv(path_dict['train_annot']+"JASCO_tp_train.csv")
    fp = pd.read_csv(path_dict['train_annot']+"JASCO_fp_train.csv")

    # Read test files (detailed annotations) for the Robert's bank and boundary pass locations.
    jasco_test_files_roberts = pd.read_csv(path_dict['test_annot']+"jasco_roberts_bank_annot.csv", sep=',')
    jasco_test_files_roberts["filename"] = jasco_test_files_roberts["filename"]
    jasco_test_files_boundary = pd.read_csv(path_dict['test_annot']+"jasco_boundary_pass_annot.csv", sep=',')
    jasco_test_files_boundary["filename"] = jasco_test_files_boundary["filename"]

    # Compile a list of filenames present in the test files. These will be used to make sure test data is not included in the pre-processed training database.
    all_jasco_test_files = np.concatenate([jasco_test_files_roberts.loc[:, "filename"].unique(),
                                           jasco_test_files_boundary.loc[:, "filename"].unique()])

    tp['date_time'] = pd.to_datetime(tp['date_time'], infer_datetime_format=True)
    fp['date_time'] = pd.to_datetime(fp['date_time'], infer_datetime_format=True)

    # Compile a list of all filescontaining validated detections at the presence/absence level.
    all_jasco_files = pd.concat([tp.loc[:, ("filename", "date_time")].groupby("filename").first().reset_index(),
                                fp.loc[:, ("filename", "date_time")].groupby("filename").first().reset_index()])

    # Determine the indices for those file that will be reserved for testing
    indices_used_for_test = all_jasco_files.index[all_jasco_files.filename.isin(all_jasco_test_files)].to_list()
    # Get the dates for those files that will be reserved for testing
    dates_used_for_test = all_jasco_files.iloc[indices_used_for_test].date_time.to_list()

    # Add a buffer of one day befor and one day after each date that contained data reserved for testing.
    # This ensures there's at least 1 day between any clip used for testing and data that is used for training.
    days_used_for_test = set([d.date() for d in dates_used_for_test])
    days_used_for_test_buffer = days_used_for_test.union({d - datetime.timedelta(days=1) for d in days_used_for_test})
    days_used_for_test_buffer = days_used_for_test_buffer.union({d + datetime.timedelta(days=1) for d in days_used_for_test})

    # Exclude any test dates (with the 1 day buffer) from the data vailable for training.
    tp_exclude_test = tp[tp.apply(func=lambda r:r['date_time'].date() not in days_used_for_test_buffer, axis=1)]
    fp_exclude_test = fp[fp.apply(func=lambda r:r['date_time'].date() not in days_used_for_test_buffer, axis=1)]

    # Standardize the annotation tables to the ketos format
    std_annot_train_tp = sl.standardize(table=tp_exclude_test, signal_labels=[1],  trim_table=True)
    std_annot_train_fp = sl.standardize(table=fp_exclude_test, signal_labels=[1], backgr_labels=[0], trim_table=True)

    print("std_annot_train_tp, std_annot_train_fp: ", len(std_annot_train_tp), len(std_annot_train_fp))

    # Create 5s clips
    std_annot_train_tp = sl.select(annotations=std_annot_train_tp,
                                   length=spec_cfg['duration'], step=select_step, 
                                   min_overlap=select_min_overlap, center=False)
    std_annot_train_fp = sl.select(annotations=std_annot_train_fp,
                                   length=spec_cfg['duration'], step=select_step, 
                                   min_overlap=select_min_overlap, center=False)  # Augmented with time-shifts

    print("std_annot_train_tp, std_annot_train_fp: ", len(std_annot_train_tp), len(std_annot_train_fp))

    # Create the database with positive and negative datasets
    create_database(output_file=path_dict['database_save_filename'], data_dir=path_dict['audio_data_dir'],
                    dataset_name='kw', selections=std_annot_train_tp,
                    audio_repres=spec_cfg)

    create_database(output_file=path_dict['database_save_filename'], data_dir=path_dict['audio_data_dir'],
                    dataset_name='other', selections=std_annot_train_fp,
                    audio_repres=spec_cfg)

In [4]:
def add_negative_samples(annot_df, map_to_ketos_annot_std, add_ratio):
    """ Add negative samples from randomly selected files from the annotation dataframe
     
        Args:
            annot_df: pandas DataFrame
                Annotation table.

        Returns:
            annot_df: pandas DataFrame
                Annotation table after pre-processing
            map_to_ketos_annot_std: dict
                Dictionary of dataframe's column name mapping to rename
            add_ratio: float
                Percentage of file to be used from the annotation dataframe for extracting negative samples

    """
    # Change the location of 'target_wav_location' to choose random neg samples from that location
    list_of_wav_files=annot_df['filename'].unique()
    print("list_of_wav_files:", len(list_of_wav_files))

    # Define the ratio of files to be taken from the folder
    total_files=int(len(list_of_wav_files)*add_ratio)
    print("total_files:", total_files)

    # Now, generate list of unique file indices
    file_indices_random = random.sample(range(0, len(list_of_wav_files)), total_files)

    total_wav_parsed = 0
    total_number_of_negative_samples=0
    for file_index in file_indices_random:
        target_wav_filename=list_of_wav_files[file_index]
        target_matched_annot_df = annot_df[annot_df['filename'].str.find(os.path.basename(target_wav_filename)) != -1]
        
        if(len(target_matched_annot_df)!=0):
            total_time_of_file = librosa.get_duration(filename=target_wav_filename)
            target_files_with_len = pd.DataFrame({'filename':[target_wav_filename], 
                                                'duration':[total_time_of_file]})

            #Standardize annotation table format
            annot, label_dict = sl.standardize(target_matched_annot_df, mapper=map_to_ketos_annot_std, return_label_dict=True, trim_table=True)

            sel = sl.select_by_segmenting(files=target_files_with_len, 
                                    length=spec_cfg['duration'], 
                                    annotations=annot, 
                                    step=spec_cfg['duration'],
                                    keep_only_empty=True) 

            # Check if there is any BH class in the selection table
            sel_only_0=sel.loc[sel['label']==0] 
            
            create_database(output_file=path_dict['database_save_filename'], data_dir=path_dict['audio_data_dir'],
                                            dataset_name='other', selections=sel_only_0,
                                            audio_repres=spec_cfg)

            total_wav_parsed+=1
            total_number_of_negative_samples+=len(sel_only_0)
    
    print("Total audio files processed: ", total_wav_parsed)
    print("Total negative samples added: ", total_number_of_negative_samples)
    
    # ============== Add Negative/Other class samples ===============
    annot_df=pd.read_csv(path_dict['train_annot']+"onc_barkley_canyon_train_multiclass.csv", sep=',')
    map_to_ketos_annot_std={'sound_id_species': 'label'}
    add_negative_samples(annot_df, map_to_ketos_annot_std, add_ratio=0.2)

In [ ]:
def create_onc_kw_hb_d_db():
    dataset_name='onc_barkley_canyon'
    annot_df=pd.read_csv(path_dict['train_annot']+"onc_barkley_canyon_train_multiclass.csv", sep=',')

    map_to_ketos_annot_std ={'sound_id_species': 'label'} 
    std_annot_df, label_dict=sl.standardize(table=annot_df,  mapper=map_to_ketos_annot_std, signal_labels=['KW', 'HB', 'D'], trim_table=False, return_label_dict=True)
    print("label_dict:", label_dict) # {'KW': 1, 'HB': 2, 'D': 3}
    
    print("std_annot_df:", len(std_annot_df))
    sel_annot_df = sl.select(annotations=std_annot_df, length=spec_cfg['duration'], step=select_step, min_overlap=select_min_overlap, center=False)
    print("sel_annot_df:", len(sel_annot_df))

    kw_sel_annot_df=sl.query(sel_annot_df, annotations=None, filename=None, label=1, start=None, end=None)
    hb_sel_annot_df=sl.query(sel_annot_df, annotations=None, filename=None, label=2, start=None, end=None)
    d_sel_annot_df=sl.query(sel_annot_df, annotations=None, filename=None, label=3, start=None, end=None)
    
    print(len(kw_sel_annot_df), len(hb_sel_annot_df), len(d_sel_annot_df))
    
    print("Adding positive samples from ", dataset_name)

    create_database(output_file=path_dict['database_save_filename'], data_dir=path_dict['audio_data_dir'],
                                dataset_name='kw', selections=kw_sel_annot_df,
                                audio_repres=spec_cfg)
    create_database(output_file=path_dict['database_save_filename'], data_dir=path_dict['audio_data_dir'],
                                dataset_name='hb', selections=hb_sel_annot_df,
                                audio_repres=spec_cfg)
    create_database(output_file=path_dict['database_save_filename'], data_dir=path_dict['audio_data_dir'],
                                dataset_name='dolphin', selections=d_sel_annot_df,
                                audio_repres=spec_cfg)

In [5]:
missing_files=['/data/HALLO/JASCO-Malahat-VFPA/audio/SHMALAHAT-STN4-2015FALL/AMAR439.9.20151015T225849Z.wav',
 '/data/HALLO/JASCO-Malahat-VFPA/audio/SHMALAHAT-STN4-2015FALL/AMAR439.9.20151015T235849Z.wav',
 '/data/HALLO/JASCO-Malahat-VFPA/audio/SHMALAHAT-STN4-2015FALL/AMAR439.9.20151016T002849Z.wav',
 '/data/HALLO/JASCO-Malahat-VFPA/audio/SHMALAHAT-STN4-2015FALL/AMAR439.9.20151016T005849Z.wav',
 '/data/HALLO/JASCO-Malahat-VFPA/audio/SHMALAHAT-STN4-2015FALL/AMAR439.9.20151024T195849Z.wav',
 '/data/HALLO/JASCO-Malahat-VFPA/audio/SHMALAHAT-STN4-2015FALL/AMAR439.9.20151024T205849Z.wav',
 '/data/HALLO/JASCO-Malahat-VFPA/audio/SHMALAHAT-STN4-2015FALL/AMAR439.9.20151024T211349Z.wav',
 '/data/HALLO/JASCO-Malahat-VFPA/audio/SHMALAHAT-STN4-2015FALL/AMAR439.9.20151027T235849Z.wav',
 '/data/HALLO/JASCO-Malahat-VFPA/audio/SHMALAHAT-STN4-2015FALL/AMAR439.9.20151110T065849Z.wav',
 '/data/HALLO/JASCO-Malahat-VFPA/audio/SHMALAHAT-STN4-2015FALL/AMAR439.9.20151110T071349Z.wav',
 '/data/HALLO/JASCO-Malahat-VFPA/audio/SHMALAHAT-STN4-2015FALL/AMAR439.9.20151110T072849Z.wav',
 '/data/HALLO/JASCO-Malahat-VFPA/audio/SHMALAHAT-STN4-2015FALL/AMAR439.9.20151110T074349Z.wav',
 '/data/HALLO/JASCO-Malahat-VFPA/audio/SHMALAHAT-STN4-2015FALL/AMAR439.9.20151114T021349Z.wav']

def create_jasco_malahat_vfpa_kw_hb_other_db():
    dataset_name='jasco_malahat_vfpa'
    annot_df=pd.read_csv(path_dict['train_annot']+"jasco_malahat_vfpa_train_multiclass.csv", sep=',')

    # ======== Removing missing files ========
    # annot_df = annot_df[annot_df.filename in missing_files]
    # ======== Removing missing files ========

    std_annot_df, label_dict=sl.standardize(table=annot_df, signal_labels=['kw', 'hb', 'other'], trim_table=False, return_label_dict=True)
    print("label_dict:", label_dict)
    # {'kw': 1, 'hb': 2, 'other': 3}

    sel_annot_df = sl.select(annotations=std_annot_df, length=spec_cfg['duration'], step=select_step, min_overlap=select_min_overlap, center=False)
    print("sel_annot_df:", len(sel_annot_df))
    kw_sel_annot_df=sl.query(sel_annot_df, annotations=None, filename=None, label=1, start=None, end=None)
    hb_sel_annot_df=sl.query(sel_annot_df, annotations=None, filename=None, label=2, start=None, end=None)
    other_sel_annot_df=sl.query(sel_annot_df, annotations=None, filename=None, label=3, start=None, end=None)

    print(len(kw_sel_annot_df), len(hb_sel_annot_df), len(other_sel_annot_df))

    print("Adding positive samples from ", dataset_name)

    create_database(output_file=path_dict['database_save_filename'], data_dir=path_dict['audio_data_dir'],
                                dataset_name='kw', selections=kw_sel_annot_df,
                                audio_repres=spec_cfg)
    create_database(output_file=path_dict['database_save_filename'], data_dir=path_dict['audio_data_dir'],
                                dataset_name='hb', selections=hb_sel_annot_df,
                                audio_repres=spec_cfg)
    create_database(output_file=path_dict['database_save_filename'], data_dir=path_dict['audio_data_dir'],
                                dataset_name='other', selections=other_sel_annot_df,
                                audio_repres=spec_cfg)

In [6]:
create_jasco_roberts_kw_db()
create_onc_kw_hb_d_db()
create_jasco_malahat_vfpa_kw_hb_other_db()